# BerTopic
based on 1st starting posts in "Kids Psychology and Development" Russian women forum eva.ru

In [ ]:
!pip install bertopic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from bertopic import BERTopic

In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/'

Messages_filename='137.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)

Messages_topics_filename='137Topics.csv'
Messages_topics_full_filename=os.path.join(Data, Messages_topics_filename)

text='message'


Models = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Models/ChapterTopics/'
ModelName='137'


In [5]:
df=pd.read_csv(Messages_full_filename, error_bad_lines=False, index_col=False)

In [6]:
#change text column type to string
df[text]=df[text].values.astype(str)

In [7]:
import nltk

In [79]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [80]:
nltk.download('stopwords')
from nltk.corpus import stopwords
ru_stopwords = stopwords.words('russian')
eng_stopwords = stopwords.words('english')
add_stopwords = ['ай',
'ну',
'по',
'это',
'так',
'что',
'вот',
'кого',
'то',
'кто',
'все',
'мочь',
'всё',
'ваш',
'спасибо',
'пожалуйста',
'посоветуйте',
'вопрос',
'весь',
'ав',
'очень',
'ещё',
'просто',
'свой',
'ребёнок',
'сын',
'девочка',
'мальчик',
'дочка',
'делать',
'дочь',
'ребёночек',
'собственно',
'сабж',
'согласный',
'ть',
'подсказать',
'раздел',
'тема',
'ред'
	]
stopwords = eng_stopwords + ru_stopwords + add_stopwords

from collections import Counter
stopwords_dict = Counter(stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
#!pip install pymorphy2

In [82]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return  ' '.join(res)
def text_preprocess(row):
  s=lemmatize(row[text])
  return ' '.join([word for word in tokenizer.tokenize(s) if word not in stopwords_dict])

In [83]:
df['preprocessed_message']=df.apply(text_preprocess,axis=1)

In [84]:
df.to_csv(Messages_full_filename, header=True, index=False)

In [85]:
data = df['preprocessed_message'].tolist()
#model = BERTopic(embedding_model='blinoff/roberta-base-russian-v0')
model = BERTopic(language="Russian", calculate_probabilities=True, nr_topics=25)
topics, probabilities = model.fit_transform(data)

In [86]:
Model_full_filename=os.path.join(Models, ModelName)
model.save(Model_full_filename)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [87]:
model.visualize_topics()

In [88]:
topics_df=model.get_topic_info()

In [89]:
topics_df

,Topic,Count,Name
0,-1,15208,-1_год_хотеть_говорить_мама
1,0,1243,0_спать_сон_засыпать_ночь
2,1,778,1_читать_книга_чтение_книжка
3,2,531,2_психолог_посоветовать_хороший_психолога
4,3,426,3_русский_класс_школа_писать
5,4,278,4_звук_логопед_говорить_слово
6,5,274,5_игрушка_кукла_игра_играть
7,6,263,6_сад_группа_ходить_год
8,7,230,7_класс_учитель_школа_учительница
9,8,225,8_рисовать_цвет_рисунок_карандаш


In [90]:
num_words=4
for i in range(0,num_words):
    topics_df['w'+str(i)]=''
for i in range(0,num_words):  
    topics_df['p'+str(i)]=0.0000
for index,row in topics_df.iterrows():
    topic_details=model.get_topic(row['Topic'])
    for (i,td) in enumerate(topic_details):
        topics_df.at[index,'w'+str(i)]=td[0]
        topics_df.at[index,'p'+str(i)]=td[1]

In [91]:
topics_df.to_csv(Messages_topics_full_filename, header=True, index=False)

In [92]:
len(topics_df)

26

In [93]:
model.visualize_barchart()

In [94]:
model.visualize_heatmap()

In [95]:
model.visualize_hierarchy()

In [96]:
model	.visualize_term_rank()

In [98]:
df['Timestamp']=pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
timestamps = df['Timestamp'].to_list()

In [100]:
topics_over_time = model.topics_over_time(docs=data, 
                                                topics=topics, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [104]:
model.visualize_topics_over_time(topics_over_time, top_n_topics=5)